In [1]:
import findspark
findspark.init()

In [2]:
# winutils.exe  chmod 777  C:\spark-temp

from pyspark.conf import SparkConf
conf = SparkConf()


# config.setMaster("spark://192.168.11.71:7077").setAppName("HiveApp")
(
    conf.setMaster("local[1]").setAppName("HiveApp")
    .set("spark.executor.memory", "4G")
    .set("spark.executor.cores", 2)
    .set("spark.cores.max", 2)
    .set("spark.driver.memory", "4G")

    .set("hive.metastore.uris", "thrift://192.168.93.128:9083")

    # .set("spark.sql.warehouse.dir", "/home/ubuntu/spark-warehose")
    .set("spark.sql.warehouse.dir", "hdfs://192.168.93.128:9000/user/hive/warehouse")  # spark warehouse

    # .set("hive.metastore.warehouse.dir", "hdfs://192.168.93.128:9000/user/hive/warehouse")  # ?
    # .set("spark.local.dir", "c:/spark-temp")
)


from pyspark.sql import SparkSession, HiveContext
ss = (
    SparkSession.builder
    .config(conf=conf)
    .enableHiveSupport()
    .getOrCreate()
)

In [3]:
ss.sql("DROP TABLE IF EXISTS hive_table")
ss.sql("""
CREATE TABLE IF NOT EXISTS hive_table(number int, Ordinal_Number string, Cardinal_Number string)
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
LINES TERMINATED BY '\n'
""")

DataFrame[]

In [4]:
ss.sql("SHOW DATABASES").show()

+------------+
|databaseName|
+------------+
|     default|
|   orders_db|
+------------+



In [5]:
ss.sql("CREATE TABLE IF NOT EXISTS src(key INT, value STRING) USING hive")

DataFrame[]

In [6]:
ss.sql("INSERT INTO TABLE src values(1, 'ONE')")
ss.sql("SELECT * FROM src").show()

+---+-----+
|key|value|
+---+-----+
|  1|  ONE|
|  1|  ONE|
|  1|  ONE|
|  1|  ONE|
|  1|  ONE|
|  1|  ONE|
+---+-----+



In [7]:
ss.sql("SHOW TABLES").show()

+--------+-------------+-----------+
|database|    tableName|isTemporary|
+--------+-------------+-----------+
| default|        brand|      false|
| default|   brand_test|      false|
| default|    employees|      false|
| default|employees_ext|      false|
| default|   hive_table|      false|
| default|          src|      false|
+--------+-------------+-----------+



---

In [8]:
products = (
    # (product_id, product_name, brand_id)  
    (1, 'iPhone', 100),
    (2, 'Galaxy', 200),
    (3, 'RedMi', 300),  # orphan record, no matching brand
    (4, 'Pixel', 400),
)
brands = (
    #(brand_id, brand_name)
    (100, "Apple"),
    (200, "Samsung"),
    (400, "Google"),
    (500, "Sony"),  # no matching product
)
 
df_product = ss.createDataFrame(data=products, schema=("product_id", "product_name", "brand_id"))
df_brand = ss.createDataFrame(data=brands, schema=("brand_id", "brand_name"))

df_product.show(), df_brand.show()

+----------+------------+--------+
|product_id|product_name|brand_id|
+----------+------------+--------+
|         1|      iPhone|     100|
|         2|      Galaxy|     200|
|         3|       RedMi|     300|
|         4|       Pixel|     400|
+----------+------------+--------+

+--------+----------+
|brand_id|brand_name|
+--------+----------+
|     100|     Apple|
|     200|   Samsung|
|     400|    Google|
|     500|      Sony|
+--------+----------+



(None, None)

In [9]:
ss.sql("CREATE DATABASE IF NOT EXISTS products_db")

DataFrame[]

In [12]:
ss.sql("CREATE TABLE dummy(id Int, name String) USING hive")

DataFrame[]

In [13]:
ss.sql("SELECT * FROM dummy")

DataFrame[id: int, name: string]

In [11]:
"""
WRITE DF into DB table
"""


"""
Writes to `default` DB in the metastore_db --> (Writes to C:/spark/brand_test/ (.parquet files))
"""
df_brand.write.mode("overwrite").saveAsTable("brand_test")

"""
Writes to the specified database --> (Writes to C:/spark/product_db/brand/ (.parquet files))
Creates table in Hive's metastore_db
"""
df_brand.write.mode("overwrite").saveAsTable("brands")